# Starlink: Latency over Single Days

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import matplotlib.pyplot as plt
import pycountry
import os

def country_name(country_iso):
    return pycountry.countries.get(alpha_2=country_iso).name

In [ ]:
probes = pd.read_parquet("./parquet/ripe_atlas_probe_data.parquet").set_index('id')
probes

In [ ]:
tls_data = pd.read_parquet("./parquet/tls_data.parquet").join(probes, on='prb_id')
print(len(tls_data))
tls_data

In [ ]:
results_dir = './individual_days_results'
def filter_for(df, name, value):
    return df.loc[df[name] == value]

def filter_for_range(df, name, lower_bound, upper_bound):
    return df[(df[name] >= lower_bound) & (df[name] < upper_bound)]

def plot(hours, title, country):
    # Create a figure and axis
    fig, ax = plt.subplots()

    # Plot the boxplots
    ax.boxplot(hours, showfliers=False)

    # Set axis labels and title
    ax.set_xlabel('i-th Hour of the Day')
    ax.set_ylabel('Latencies in ms')
    ax.set_title('Boxplot of Tuples')
    plt.title(title)
    plt.savefig(results_dir + "/" + country + '/' + title.lower().replace(" ", "_") + ".pdf")
    plt.clf()

one_hour = timedelta(hours=1)
one_day = timedelta(days=1)

def plot_all_days(country):
    german_df = filter_for(tls_data, 'country', country)
    print(len(german_df))
    
    min = datetime.fromtimestamp(np.min(german_df['timestamp'].tolist()))
    max = datetime.fromtimestamp(np.max(german_df['timestamp'].tolist()))
    
    def nicify_day(d):
        return str(d.day) + "." + str(d.month) + "." + str(d.year)
    
    current = datetime(min.year, min.month, min.day)
    while current < max:
        hours = [ filter_for_range(german_df, 'timestamp', datetime.timestamp(current + (one_hour*i)), datetime.timestamp(current + (one_hour*(i+1))))['rt'] for i in range(0, 24)]
        plot(hours, "Latency Plot for " + country_name(country) +  " on " + nicify_day(current), country)
    
        current = current + one_day

if not os.path.isdir(results_dir): os.mkdir(results_dir)
countries = ['DE', 'US']
for c in countries:
    dir = os.path.join(results_dir, c)
    if not os.path.isdir(dir):
        os.mkdir(dir)
    plot_all_days(c)